In [ ]:
!nvidia-smi

In [ ]:
#installing pip packages for pretrained models
!pip install pretrainedmodels
#importing packages
import os,time
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import pretrainedmodels
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
#Checking if GPU is available
use_cuda = torch.cuda.is_available()
print("GPU available: ",use_cuda)

class FCWithLogSigmoid(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(FCWithLogSigmoid, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
        self.logsigmoid = nn.LogSigmoid()
    def forward(self, x):
        return self.logsigmoid(self.linear(x))
    

#Specifying senet model
model_name = 'senet154' # could be fbresnet152 or inceptionresnetv2
model_senet = pretrainedmodels.__dict__[model_name](pretrained=None) #pretrained='imagenet+background'

for param in model_senet.parameters():
    param.requires_grad = False

model_senet.last_linear = FCWithLogSigmoid(2048, 251)

model_senet.load_state_dict(torch.load('../input/models-v2/pytorch_senet.pt'))

#Specifying pnasnet model
model_name = 'pnasnet5large'
model_pnasnet = pretrainedmodels.__dict__[model_name](pretrained=None) #pretrained='imagenet+background'

for param in model_pnasnet.parameters():
    param.requires_grad = False

model_pnasnet.last_linear = FCWithLogSigmoid(4320, 251)
    
model_pnasnet.load_state_dict(torch.load('../input/models-v2/pytorch_pnasnet.pt'))

#Specifying polynet model
model_name = 'polynet' # could be fbresnet152 or inceptionresnetv2
model_polynet = pretrainedmodels.__dict__[model_name](pretrained=None) #pretrained='imagenet+background'

for param in model_polynet.parameters():
    param.requires_grad = False

model_polynet.last_linear = FCWithLogSigmoid(2048, 251)
    
model_polynet.load_state_dict(torch.load('../input/models-aug/pytorch_polynet.pt'))

#Specifying densenet201 model
model_densenet = models.densenet201(pretrained=False)

for param in model_densenet.parameters():
    param.requires_grad = False

model_densenet.classifier = FCWithLogSigmoid(1920, 251)
    
    
model_densenet.load_state_dict(torch.load('../input/densenet-v2/pytorch_densenet201.pt'))


if use_cuda:
    model_senet = model_senet.cuda()
    model_pnasnet = model_pnasnet.cuda()
    model_polynet = model_polynet.cuda()
    model_densenet = model_densenet.cuda()

In [ ]:
!nvidia-smi

In [ ]:
labels = {0: '0', 1: '1', 2: '10', 3: '100', 4: '101', 5: '102', 6: '103', 7: '104', 8: '105', 9: '106', 10: '107', 11: '108', 12: '109', 13: '11', 14: '110', 15: '111', 16: '112', 17: '113', 18: '114', 19: '115', 20: '116', 21: '117', 22: '118', 23: '119', 24: '12', 25: '120', 26: '121', 27: '122', 28: '123', 29: '124', 30: '125', 31: '126', 32: '127', 33: '128', 34: '129', 35: '13', 36: '130', 37: '131', 38: '132', 39: '133', 40: '134', 41: '135', 42: '136', 43: '137', 44: '138', 45: '139', 46: '14', 47: '140', 48: '141', 49: '142', 50: '143', 51: '144', 52: '145', 53: '146', 54: '147', 55: '148', 56: '149', 57: '15', 58: '150', 59: '151', 60: '152', 61: '153', 62: '154', 63: '155', 64: '156', 65: '157', 66: '158', 67: '159', 68: '16', 69: '160', 70: '161', 71: '162', 72: '163', 73: '164', 74: '165', 75: '166', 76: '167', 77: '168', 78: '169', 79: '17', 80: '170', 81: '171', 82: '172', 83: '173', 84: '174', 85: '175', 86: '176', 87: '177', 88: '178', 89: '179', 90: '18', 91: '180', 92: '181', 93: '182', 94: '183', 95: '184', 96: '185', 97: '186', 98: '187', 99: '188', 100: '189', 101: '19', 102: '190', 103: '191', 104: '192', 105: '193', 106: '194', 107: '195', 108: '196', 109: '197', 110: '198', 111: '199', 112: '2', 113: '20', 114: '200', 115: '201', 116: '202', 117: '203', 118: '204', 119: '205', 120: '206', 121: '207', 122: '208', 123: '209', 124: '21', 125: '210', 126: '211', 127: '212', 128: '213', 129: '214', 130: '215', 131: '216', 132: '217', 133: '218', 134: '219', 135: '22', 136: '220', 137: '221', 138: '222', 139: '223', 140: '224', 141: '225', 142: '226', 143: '227', 144: '228', 145: '229', 146: '23', 147: '230', 148: '231', 149: '232', 150: '233', 151: '234', 152: '235', 153: '236', 154: '237', 155: '238', 156: '239', 157: '24', 158: '240', 159: '241', 160: '242', 161: '243', 162: '244', 163: '245', 164: '246', 165: '247', 166: '248', 167: '249', 168: '25', 169: '250', 170: '26', 171: '27', 172: '28', 173: '29', 174: '3', 175: '30', 176: '31', 177: '32', 178: '33', 179: '34', 180: '35', 181: '36', 182: '37', 183: '38', 184: '39', 185: '4', 186: '40', 187: '41', 188: '42', 189: '43', 190: '44', 191: '45', 192: '46', 193: '47', 194: '48', 195: '49', 196: '5', 197: '50', 198: '51', 199: '52', 200: '53', 201: '54', 202: '55', 203: '56', 204: '57', 205: '58', 206: '59', 207: '6', 208: '60', 209: '61', 210: '62', 211: '63', 212: '64', 213: '65', 214: '66', 215: '67', 216: '68', 217: '69', 218: '7', 219: '70', 220: '71', 221: '72', 222: '73', 223: '74', 224: '75', 225: '76', 226: '77', 227: '78', 228: '79', 229: '8', 230: '80', 231: '81', 232: '82', 233: '83', 234: '84', 235: '85', 236: '86', 237: '87', 238: '88', 239: '89', 240: '9', 241: '90', 242: '91', 243: '92', 244: '93', 245: '94', 246: '95', 247: '96', 248: '97', 249: '98', 250: '99'}
order_val = list(labels.values())
order_keys = list(labels.keys())
order = [order_val.index(str(i)) for i in order_keys]

In [ ]:
#Making predictions on test data by making batches manually
from PIL import Image
import random,cv2
val_path = "../input/ifood-2019-fgvc6/test_set/test_set/"
batch_size = 64
val_images = os.listdir(val_path)
images_complete = val_images[:(len(os.listdir(val_path))//batch_size)*batch_size]
images_partial = val_images[(len(os.listdir(val_path))//batch_size)*batch_size:]

transform_senet = transforms.Compose([transforms.CenterCrop(224),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])
transform_pnasnet = transforms.Compose([transforms.CenterCrop(331),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])

transform_polynet = transforms.Compose([transforms.CenterCrop(331),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])

transform_densenet = transforms.Compose([transforms.CenterCrop(224),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])

  

data_senet = []
data_pnasnet = []
data_polynet = []
data_densenet = []
image_names = []
for i in tqdm(range(len(images_complete))):
    image = Image.open(val_path+images_complete[i])
    
    image_senet = transform_senet(image).float()
    image_senet = image_senet.unsqueeze(0)
    
    image_pnasnet = transform_pnasnet(image).float()
    image_pnasnet = image_pnasnet.unsqueeze(0)
    
    image_polynet = transform_polynet(image).float()
    image_polynet = image_polynet.unsqueeze(0)
    
    image_densenet = transform_densenet(image).float()
    image_densenet = image_densenet.unsqueeze(0)
    
    #print(image.shape)
    data_senet.append(image_senet)
    data_pnasnet.append(image_pnasnet)
    data_polynet.append(image_polynet)
    data_densenet.append(image_densenet)
    image_names.append(images_complete[i])

    if (len(data_senet) == batch_size):
        t = 0
        batch_index = []
        batch_index_top3 = []
        first_tensor_data_senet = data_senet[t]
        first_tensor_data_pnasnet = data_pnasnet[t]
        first_tensor_data_polynet = data_polynet[t]
        first_tensor_data_densenet = data_densenet[t]
        while(t<(batch_size-1)):
            second_tensor_data_senet = data_senet[t+1]
            second_tensor_data_pnasnet = data_pnasnet[t+1]
            second_tensor_data_polynet = data_polynet[t+1]
            second_tensor_data_densenet = data_densenet[t+1]
            
            data_tensor_senet = torch.cat((first_tensor_data_senet,second_tensor_data_senet),dim=0)
            data_tensor_pnasnet = torch.cat((first_tensor_data_pnasnet,second_tensor_data_pnasnet),dim=0)
            data_tensor_polynet = torch.cat((first_tensor_data_polynet,second_tensor_data_polynet),dim=0)
            data_tensor_densenet = torch.cat((first_tensor_data_densenet,second_tensor_data_densenet),dim=0)
  
            
            first_tensor_data_senet = data_tensor_senet
            first_tensor_data_pnasnet = data_tensor_pnasnet
            first_tensor_data_polynet = data_tensor_polynet
            first_tensor_data_densenet = data_tensor_densenet
            
            t+=1
        if use_cuda:
            data_tensor_senet = data_tensor_senet.cuda()
            data_tensor_pnasnet = data_tensor_pnasnet.cuda()
            data_tensor_polynet = data_tensor_polynet.cuda()
            data_tensor_densenet = data_tensor_densenet.cuda()
        #print(data_tensor.shape)
        output_senet = model_senet(data_tensor_senet)
        output_pnasnet = model_pnasnet(data_tensor_pnasnet)
        output_polynet = model_polynet(data_tensor_polynet)
        output_densenet = model_densenet(data_tensor_densenet)
        
        arr_densenet = output_densenet.data.cpu().numpy()
        res_densenet = []
        for i in range(arr_densenet.shape[0]):
            res_densenet.append([arr_densenet[i][j] for j in order])
        index_densenet = res_densenet
        index_densenet=np.array(index_densenet)
        
        #print(output.shape)
        index_senet = output_senet.data.cpu().numpy()
        index_pnasnet = output_pnasnet.data.cpu().numpy()  
        index_polynet = output_polynet.data.cpu().numpy()
        #index_densenet = output_densenet.data.cpu().numpy()
        
        index = np.add(index_senet,index_pnasnet)
        index = np.add(index,index_polynet)
        index = np.add(index,index_densenet)
        index /=4
        for k in index:
            mini_batch_index = k.argsort()[::-1][:3]
            batch_index.append(mini_batch_index)
            
        assert len(image_names) == len(batch_index)
        
        with open("pytorch_ensembling_4models.txt","a") as f:
            for l in range(len(image_names)):
                f.write(image_names[l]+','+str(batch_index[l][0])+' '+str(batch_index[l][1])+' '+
                        str(batch_index[l][2])+'\n')
        data_senet = []
        data_pnasnet = []
        data_polynet = []
        data_densenet = []
        image_names = []

data_senet = []
data_pnasnet = []
data_polynet = []
data_densenet = []
image_names = []
for i in tqdm(range(len(images_partial))):
    image = Image.open(val_path+images_partial[i])
    image_senet = transform_senet(image).float()
    image_senet = image_senet.unsqueeze(0)
    
    image_pnasnet = transform_pnasnet(image).float()
    image_pnasnet = image_pnasnet.unsqueeze(0)
    
    image_polynet = transform_polynet(image).float()
    image_polynet = image_polynet.unsqueeze(0)
    
    image_densenet = transform_densenet(image).float()
    image_densenet = image_densenet.unsqueeze(0)
    
    #print(image.shape)
    data_senet.append(image_senet)
    data_pnasnet.append(image_pnasnet)
    data_polynet.append(image_polynet)
    data_densenet.append(image_densenet)
    image_names.append(images_partial[i])

    if (len(data_senet) == len(images_partial)):
        t = 0
        batch_index = []
        batch_index_top3 = []
        first_tensor_data_senet = data_senet[t]
        first_tensor_data_pnasnet = data_pnasnet[t]
        first_tensor_data_polynet = data_polynet[t]
        first_tensor_data_densenet = data_densenet[t]
        
        while(t<(len(images_partial)-1)):
            second_tensor_data_senet = data_senet[t+1]
            second_tensor_data_pnasnet = data_pnasnet[t+1]
            second_tensor_data_polynet = data_polynet[t+1]
            second_tensor_data_densenet = data_densenet[t+1]
            
            data_tensor_senet = torch.cat((first_tensor_data_senet,second_tensor_data_senet),dim=0)
            data_tensor_pnasnet = torch.cat((first_tensor_data_pnasnet,second_tensor_data_pnasnet),dim=0)
            data_tensor_polynet = torch.cat((first_tensor_data_polynet,second_tensor_data_polynet),dim=0)
            data_tensor_densenet = torch.cat((first_tensor_data_densenet,second_tensor_data_densenet),dim=0)
    
            
            first_tensor_data_senet = data_tensor_senet
            first_tensor_data_pnasnet = data_tensor_pnasnet
            first_tensor_data_polynet = data_tensor_polynet
            first_tensor_data_densenet = data_tensor_densenet
            
            t+=1
        if use_cuda:
            data_tensor_senet = data_tensor_senet.cuda()
            data_tensor_pnasnet = data_tensor_pnasnet.cuda()
            data_tensor_polynet = data_tensor_polynet.cuda()
            data_tensor_densenet = data_tensor_densenet.cuda()
        #print(data_tensor.shape)
        output_senet = model_senet(data_tensor_senet)
        output_pnasnet = model_pnasnet(data_tensor_pnasnet)
        output_polynet = model_polynet(data_tensor_polynet)
        output_densenet = model_densenet(data_tensor_densenet)
        
        arr_densenet = output_densenet.data.cpu().numpy()
        res_densenet = []
        for i in range(arr_densenet.shape[0]):
            res_densenet.append([arr_densenet[i][j] for j in order])
        index_densenet = res_densenet
        index_densenet=np.array(index_densenet)
        
        #print(output.shape)
        index_senet = output_senet.data.cpu().numpy()
        index_pnasnet = output_pnasnet.data.cpu().numpy()
        index_polynet = output_polynet.data.cpu().numpy()
        #index_densenet = output_densenet.data.cpu().numpy()
        
        index = np.add(index_senet,index_pnasnet)
        index = np.add(index,index_polynet)
        index = np.add(index,index_densenet)
        index /=4
        for k in index:
            mini_batch_index = k.argsort()[::-1][:3]
            batch_index.append(mini_batch_index)
            
        assert len(image_names) == len(batch_index)
        
        with open("pytorch_ensembling_4models.txt","a") as f:
            for l in range(len(image_names)):
                f.write(image_names[l]+','+str(batch_index[l][0])+' '+str(batch_index[l][1])+' '+
                        str(batch_index[l][2])+'\n')
        data_senet = []
        data_pnasnet = []
        data_polynet = []
        data_densenet = []
        image_names = []

In [ ]:
from IPython.display import FileLink
FileLink('pytorch_ensembling_4models.txt')